In [1]:
import tensorflow as tf
from tensorflow import keras

2022-03-30 16:28:06.296324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benson516/agv_ws/devel/lib:/opt/ros/noetic/lib
2022-03-30 16:28:06.296344: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Managing the GPU RAM

In [2]:
'''
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_1.py
'''

'\n$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py\n$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_1.py\n'

In [3]:
# # Set memory limit to GPUs
# for gpu in tf.config.experimental.list_physical_devices("GPU"):
#     tf.config.experimental.set_virtual_device_configuration(
#         gpu,
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

In [4]:
# # Runtime grow memory allocation
# for gpu in tf.config.experimental.list_physical_devices("GPU"):
#     tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
# Split into virtual GPUs
physical_gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
            physical_gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
             tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

2022-03-30 16:28:07.221935: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-03-30 16:28:10.639804: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-30 16:28:10.639824: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benson516-P1G2): /proc/driver/nvidia/version does not exist


IndexError: list index out of range

## Placing Operations and Variables on Devices

In [6]:
a = tf.Variable(42.0)

2022-03-30 16:28:22.261115: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
a.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [8]:
b = tf.Variable(42)

In [9]:
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [10]:
# Set the debug log to show the device placement
tf.debugging.set_log_device_placement(True)

In [11]:
c = tf.Variable(42.0)

Place on other device

In [12]:
with tf.device("/cpu:0"):
    c = tf.Variable(42.0)

In [13]:
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'